# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
This is a classification problem. In this case we have a labeled training dataset in which the end result of a student (passed or not passed) is given. Since we only want to predict whether or not the student passes, and not some continuous variable like an exam score or a number of absences, classification algorithms will suffice. Furthermore, we can use any two-class classification algorithm since the feature 'passed' has only two levels.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [2]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [3]:
student_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes


In [10]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]

# TODO: Calculate passing students
n_passed = student_data[student_data.passed == 'yes'].shape[0]

# TODO: Calculate failing students
n_failed = student_data[student_data.passed == 'no'].shape[0]

# TODO: Calculate graduation rate
grad_rate = 100.0*n_passed/n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [13]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1]

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [42]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():

        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [45]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train
# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all,y_all, random_state=0, test_size=num_test, train_size=num_train)
# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **
* Logistic Regression
    * Logistic regression is generally applied to classificiation problems and provides an output prediction per example that is bounded by 0 and 1, and can be thus be interpreted as a probability. A threshold can then be set to assign a predicted class as per that prediction. Typically a strength of logistic regression is that it provides this probabilistic interpretation, and that one can interpret the magnitude and sign of the model coefficients to give insight to the most important features in the model. Additionally, logistic regression provides handles to avoid overtraining through regularization parameters. Possible weaknesses are that it assumes one only has two levels in the target variable, and if we had more than two levels the prediction may be worse. Another weakness of a naive application of logistic regression is that the model assumes no interactions between the input features, unless we specifically code in additional features that are functions of the initial feature set. This may result in a model that has greater bias and less variance. I chose to apply Logistic regression because of the above strengths


* Random Forest
    * Random Forests are an extension of decision trees and can be generally used either for regression or classification. One strength of Random Forests is that the tree structure of the model allows one to capture elements of non-linearity in the features that may result in better performance. Another strength is that the random nature of the algorithm in selecting among subsets training examples per tree in the forest and among subsets of features at each split in the tree will help the model avoid overfitting and thus perform better on held-out test data. Additionally, Random Forests make no assumptions about the underlying distributions of the individual features (compared with, e.g. linear regression). A possible weakness here is that each split in the tree for a categorical variable will choose among the levels for that variable, and since we've used one-hot encoding for the categorical variables, the algorithm will be searching for good splits in a more sparse feature space. Another weakness is that it is somewhat more difficult to intuitively extract feature importances from Random Forests. Although we can get a magnitude of the feature's importance, the contribution of one feature to the final prediction and the directionality of how that feature impacts the final prediction are more difficult to obtain than in the logistic regression case. Finally, another potential weakness may be that if we had only a few important features and many unimportant features, we may get unlucky and create many trees that were poorly performing, reducing our performance. I chose to apply Random Forest as a quick test to see of capturing non-linearities would yield a better result (compared to Logistic regression).

* K-nearest neighbors
    * K-nearest neighbors is another algorithm that may be used for either regression or classification. Strengths of K-nearest neighbors are that there is very low training time since we do not actually construct a model, we just look up distances to K-neighbors and evaluate a vote. The lack of a parametric model yields another strength, which is that KNN is somewhat robust to outliers. One weakness is that for each point we want to serve a prediction for, we have to scan the entire dataset to find the K-closest points. In contrast, the two other algorithms above are trained once, and then for each prediction point we just have to evaluate a trained model. This could however be seen as a strength or a weakness depending on the speficic context of how we intend to use the algorithm. Another weakness is that we gain no intuitive understanding or parametric model to aid our understanding of the situation, we just issue predictions but do not learn anything about the underlying processes that cause an example to fall into a certain category. I chose to apply KNN as it may be the most naive model to try in this case, and it may serve as a good baseline. Additionally the data are small, predicting on held-out data will not be an issue.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [46]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [47]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# TODO: Initialize the three models
clf_A = KNeighborsClassifier(n_neighbors=5)
clf_B = RandomForestClassifier(n_estimators=10)
clf_C = LogisticRegression(C=1)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
for clf in [clf_A, clf_B, clf_C]:
    train_predict(clf, X_train_100, y_train_100, X_test, y_test)
    train_predict(clf, X_train_200, y_train_200, X_test, y_test)
    train_predict(clf, X_train_300, y_train_300, X_test, y_test)

Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0048 seconds
Made predictions in 0.0021 seconds.
F1 score for training set: 0.7972.
Made predictions in 0.0013 seconds.
F1 score for test set: 0.7068.
Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.0007 seconds
Made predictions in 0.0036 seconds.
F1 score for training set: 0.8571.
Made predictions in 0.0016 seconds.
F1 score for test set: 0.7121.
Training a KNeighborsClassifier using a training set size of 300. . .
Trained model in 0.0009 seconds
Made predictions in 0.0050 seconds.
F1 score for training set: 0.8722.
Made predictions in 0.0019 seconds.
F1 score for test set: 0.7482.
Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.0306 seconds
Made predictions in 0.0014 seconds.
F1 score for training set: 0.9844.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.6462.
Training a RandomForestClassifier usin

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - K-Nearest Neighbors**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.0021         |       0.0013           |    0.7972        |0.7068           |
| 200               |        0.0036           |          0.0016        |     0.8571       |0.7121           |
| 300               |            0.0050       |        0.0019          |    0.8722        |    0.7482       |

** Classifer 2 - Random Forest**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |           0.0014        |             0.0010     |  0.9844          |      0.6462     |
| 200               |     0.0011              |           0.0008       | 0.9887           |     0.7681      |
| 300               |       0.0011            |          0.0007        |     0.9927       |     0.7481      |

** Classifer 3 - Logistic Regression**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |           0.0086        |         0.0003         |       0.8571     |       0.7612    |
| 200               |         0.0024          |        0.0002          |  0.8380          |    0.7794       |
| 300               |          0.0002         |            0.0002      |   0.8381         |   0.7910        |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
Given the available data, limited resources, and cost, we should pursue logistic regression as the best model. Logistic regression, without any additional work, provides the best performance on the test data set compared to the other models. Importantly, the performance of the model on the training set is closer to the performance on the testing set than for all the other models, which means we have a better chance that the logistic model will perform well on other students which we have not evaluated yet. 
Additionally, Logistic Regression will help us use our limited resources better. Logistic Regression gives us a score for each student rather than only a binary prediction of passing, and we can use this score to rank students by risk of failure, and then prioritize our resources accordingly. Choosing Logistic Regression will also give us the added benefit of insight into which characteristics of the students are most influential as to whether they will pass or fail, so we can use this information to give specific advice in our interventions.
Looking to the future, if we were to have more data available, it may be a good idea to continue exploring the Random Forest model, but if we are limited by both traning data and time to complete the analysis, this model currently overfits the data, which means it does not perform well on students outside of the training data. In short, we should avoid the Random Forest model in the short-term and stick with the Logistic Regression.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **
At a high level, logistic regression aims to predict a specific attribute such as passed, or not passed, for a student. Logistic regression will take our features for each student as an input, and then output a score between 0 and 1, where in our case we'll say "not passed" is 0, and "passed" is 1. Since we have many features for each student, logistic regression multiplies each of those features by a weight, and attempts to find the weights for each feature that give us the best results on our prediction task. Once we've found those weights for each feature, we can make a prediction for any student by multiplying their features by the appropriate weights, and evaluating the logistic function for that combination of weights and features, which will give us the score between 0 and 1.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [56]:
# TODO: Import 'gridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score,make_scorer

# TODO: Create the parameters list you wish to tune
parameters = {'C' : [.01,.1,1,10,100],'penalty': ['l1','l2'], 'class_weight' : [None, 'balanced']}

# TODO: Initialize the classifier
clf = LogisticRegression(C=1)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters,scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

print grid_obj.best_params_
# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

{'penalty': 'l1', 'C': 0.1, 'class_weight': None}
Made predictions in 0.0004 seconds.
Tuned model has a training F1 score of 0.8267.
Made predictions in 0.0002 seconds.
Tuned model has a testing F1 score of 0.7917.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*


**Answer: **
* For the final model, the performance in the training set is ~1% worse than the untuned model, and has nearly the same performance on the test set. The reason for this is probably twofold:
    * 1) The tuning is focused on optimizing regularization parameters, which probably would not improve performance so much as it would limit overtraining, and we already chose logistic regression because the untuned model seemed the least overtrained.
    * 2) The other parameter we do a grid search over is whether or not to do class weighting to account for class imbalance when calculating the cost function, and since the classes are not heavily imbalanced, we don't seem to gain much here
    
I would have expected that tuning k for KNN or tuning the various features for the Random Forest may indeed have yielded more substantially different results than their untuned versions.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.